# ***Capstone Part 1: Segmenting and Clustering Neighborhoods in Toronto***

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

!pip3 install BeautifulSoup4
!pip install lxml
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


## Downloading Data

In [11]:
data = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M") #read page into dataframe

In [12]:
print(len(data)) #number of tables in the page

3


In [13]:
df = data[0] #used list to locate the table

In [14]:
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## Cleaning Data

In [20]:
df = df[df.Borough != 'Not assigned'] #removing cells with no boroughs

In [23]:
df = df.reset_index(drop=True) #resetting index after dropping cells

In [24]:
print(df.shape[0]) #number of rows after removing cells

103


# ***Capstone Part 2: Segmenting and Clustering Neighborhoods in Toronto***

## Adding Longitudes and Lattitudes

In [32]:
df2 = pd.read_csv('https://cocl.us/Geospatial_data') #due to errors with the geocoder package used csv to read lat & long
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [35]:
df3 = df.join(df2.set_index('Postal Code'), on='Postal Code') #joined the two data frames on postal code to get latitude and longitude for each postal code
df3.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
